## Get html content

In [63]:
import requests

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(url)

## Parse html content

In [64]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(wiki_page.content, 'html.parser')

In [65]:
for c0 in soup.contents:
    print(type(c0))


<class 'bs4.element.Doctype'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>


In [66]:
html = list(soup.children)[2]

In [67]:
for child in list(html.children):
    print(type(child))

<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>


## Extract header information

In [68]:
head = soup.find('head')

## Extract portal codes from the 1st table

In [69]:
tables = soup.findAll('table')
cells = tables[0].findAll('td')
print('Number of cells: ', len(cells))

Number of cells:  180


In [76]:
import re

pattern = re.compile(r"(?P<code>M\d[A-Z])(?P<borough>[\w\d\s\']+)(?P<nbs>.+)")

def parse(text):
    matches = pattern.search(text)
    code = matches.group('code')
    borough = matches.group('borough')
    nbs = matches.group('nbs')
    if ((borough is not None) and (nbs is None)):
        print('Exception: ', text)
        nbs = borough
    else:
        nbs = re.sub(r'\s+', ' ', nbs)
        nbs = re.sub(r'[\(\)/]', ',', nbs)
        nbs = re.sub(r'^,', '', nbs)
        nbs = re.sub(r',$', '', nbs)
    return (code, borough, nbs)

In [79]:
import pandas as pd

NEG_KEY = 'Not assigned'


pc_dict = {}
pc_dict['PostalCode'] = []
pc_dict['Borough'] = []
pc_dict['Neighborhood'] = []

for cell in cells:
    content = cell.get_text()
    if NEG_KEY in content:
        continue

    code, borough, nbs = parse(content)
    
    if ('M7Y' == code):
        print(content)
    
    pc_dict['PostalCode'].append(code)
    pc_dict['Borough'].append(borough)
    pc_dict['Neighborhood'].append(nbs)

ds = pd.DataFrame(pc_dict, columns=['PostalCode', 'Borough', 'Neighborhood'])
ds


M7YEast TorontoBusiness reply mail Processing Centre969 Eastern(Enclave of M4L)




,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


In [81]:
ds['M9W' == ds['PostalCode']]

,PostalCode,Borough,Neighborhood
94,M9W,EtobicokeNorthwest,"Clairville , Humberwood , Woodbine Downs , Wes..."
